In [38]:
import getpass
import os
import json

os.environ["OPENAI_API_KEY"] = getpass.getpass()

from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4o-mini")

In [39]:

from langchain import hub
from langchain_chroma import Chroma
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter

from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_chroma import Chroma
from langchain_openai import OpenAIEmbeddings

loader = PyPDFLoader("./chess.pdf")
docs = loader.load()

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
splits = text_splitter.split_documents(docs)

def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)


In [28]:
text = format_docs(docs)

import tiktoken
model = "gpt-4"
encoding = tiktoken.encoding_for_model(model)

tokens = encoding.encode(text)
token_count = len(tokens)

print(token_count)

5775


In [40]:
vectorstore = Chroma.from_documents(documents=splits, embedding=OpenAIEmbeddings())

retriever = vectorstore.as_retriever()
prompt = hub.pull("rlm/rag-prompt")


/home/amogha/.local/lib/python3.10/site-packages/langsmith/client.py:234: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


In [41]:
from glob import glob
with open("scene.json") as scene_file, open("scene-validator.json") as validator_file:
    scene_data = json.load(scene_file)
    validator_data = json.load(validator_file)

def load_examples(folder_path):
    example_files = glob(os.path.join(folder_path, "*.json"))
    examples = {}
    
    for file_path in example_files:
        file_name = os.path.basename(file_path).replace(".json", "")
        with open(file_path, 'r') as f:
            example_data = json.load(f)
            for field, value in example_data.items():
                if field not in examples:
                    examples[field] = []
                examples[field].append({
                    "input": f"{file_name}.",
                    "output": value
                })
    return examples

examples_folder = "./examples"
examples = load_examples(examples_folder)

def generate_question_for_field(field_name, validation_rules, examples):
    typeof = validation_rules.get("typeof", "text")
    query = validation_rules.get("query", f"Please provide the value for {field_name}")
    comment = validation_rules.get("%comment%", "")
    
    question = f"{query} (Expected type: {typeof})?"
    if comment:
        question += f" Note: {comment}"
    
    if typeof == "object":
        question = f"How many {query}? Please provide details for each. {comment}"

    if field_name in examples:
        few_shot_examples = "\n\n".join(
            f"Input: {ex['input']}\nOutput: {ex['output']}"
            for ex in examples[field_name]
        )
        question = f"Examples:\n{few_shot_examples}\n\n{question}"
    
    return question

questions = {
    field: generate_question_for_field(field, rules, examples)
    for field, rules in validator_data.items()
}

In [42]:
print(questions)

{'_scenename': 'Examples:\nInput: bowling alley.\nOutput: Bowlling Alley\n\nInput: restaurant.\nOutput: Restaurant scene\n\nInput: train.\nOutput: Train scene\n\nPlease provide the value for _scenename (Expected type: string)? Note: Mention your VR Scene name here', '_sid': 'Examples:\nInput: bowling alley.\nOutput: 45189\n\nInput: restaurant.\nOutput: scene_res\n\nInput: train.\nOutput: scene_train\n\nPlease provide the value for _sid (Expected type: string)? Note: A fillable Unique Identifier of the scene', '_slabel': 'Examples:\nInput: bowling alley.\nOutput: This is a interactive game\n\nInput: restaurant.\nOutput: This scene can be used to simulate social interactions that may trigger anxiety or paranoia in individuals with psychosis. Avatars can be used to simulate a difficult conversation with a stranger, or simulate an uncomfortable situation at the restaurant, such as a spill or a loud noise.\n\nInput: train.\nOutput: This scene can help the user get accustomed with moving in 

In [43]:
token_count = 0
for field, question in questions.items():
    tokens = encoding.encode(question)
    token_count += len(tokens)

print(token_count)

2435


In [44]:
prompt_text = """
You are tasked with generating the VR requirements for a chess game scene. Based on the given JSON structure, you should fill in the required fields by answering questions related to the environment setup, user interactions, game mechanics, user interface, and immersion enhancements. 

For each question, refer to the provided examples from prior scenes to guide your responses. These examples illustrate expected answer formats and detail levels. Use them to provide comprehensive, specific specifications that will be used to populate the scene configuration accurately.
"""

rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

responses = []
for field, question in questions.items():
    print(f"Filling field '{field}'")
    full_prompt = f"{prompt_text}\n\nQuestion: {question}"
    response = rag_chain.invoke(full_prompt)
    print(f"Answer: {response}")
    responses.append(response)


Filling field '_scenename'
Answer: Chess scene
Filling field '_sid'
Answer: scene_chess
Filling field '_slabel'
Answer: This scene represents an immersive chess game environment, where players engage in strategic gameplay on a traditional 8x8 chessboard. The board features alternating light and dark squares, with pieces positioned according to standard chess rules, allowing for competitive interaction. Players can experience both solo and multiplayer modes, enhancing social engagement and cognitive challenge in a virtual reality setting.
Filling field '#pid'
Answer: The value for #pid should be a unique identifier string that represents the play area in the VR scene. You could use a format such as "chess_playarea_001" to ensure it is distinctive. This string should be consistent with any existing naming conventions in your project.
Filling field '#length_playarea'
Answer: The value for #length_playarea should be "8.5 feet". This dimension allows for adequate space around the standard 8

In [ ]:
prompt_text = """
You are tasked with generating the VR requirements for a chess game scene. Based on the given JSON structure, you should fill in the required fields by answering questions related to the environment setup, user interactions, game mechanics, user interface, and immersion enhancements. 

For each question, refer to the provided examples from prior scenes to guide your responses. These examples illustrate expected answer formats and answer types. Use them to provide specific answers that are clear and concise, without unnecessary details. Your responses should be succinct and focus on the essential information needed to populate the scene configuration accurately.
"""

rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

responses = []
for field, question in questions.items():
    print(f"Filling field '{field}'")
    full_prompt = f"{prompt_text}\n\nQuestion: {question}"
    response = rag_chain.invoke(full_prompt)
    print(f"Answer: {response}")
    responses.append(response)


Filling field '_scenename'
Answer: Chess scene
Filling field '_sid'
Answer: scene_chess
Filling field '_slabel'
Answer: This scene represents a virtual reality chess game played on a traditional 8x8 board with alternating light and dark squares. Players can interact by moving pieces according to standard chess rules, while the immersive environment enhances strategic thinking and concentration. The setup includes visual aids for piece placement and timers for competitive play, simulating a real-world chess experience.
Filling field '#pid'
Answer: I don't know.
Filling field '#length_playarea'
Answer: The value for #length_playarea is "8".
Filling field '#breadth_playarea'
Answer: The value for #breadth_playarea is "8".
Filling field '#height_playarea'
Answer: The value for #height_playarea should be "1.5 meters". This height allows for an immersive user experience while playing chess in a virtual reality environment. It is important to ensure that players can comfortably engage with th

In [21]:
print(rag_chain.invoke('Tell me about chess'))

Chess is a two-player board game played on an 8×8 grid, involving strategic moves of sixteen pieces per player, including a king and queen. It has roots in the seventh-century Indian game chaturanga and evolved through Persia to Europe, with modern rules established by the end of the 15th century. Today, chess is governed internationally by FIDE and is one of the most popular games worldwide, with a rich body of theory and competitive history.
